In [1]:
# Import the required modules.
import os

# Paths for the data directory, text file, and compressed file.
data_directory = os.path.join('.', 'data')
if not os.path.exists(data_directory):
    print("[ERROR] data directory ('{}') does not exist".format(data_directory))

INPUT_FILE_NAME = "finefoods.txt"
OUTPUT_FILE_NAME = "finefoods.csv"

input_filepath = os.path.join(data_directory, INPUT_FILE_NAME)
csv_filepath = os.path.join(data_directory, OUTPUT_FILE_NAME)

header = [
    "product/productId",
    "review/userId",
    "review/profileName",
    "review/helpfulness",
    "review/score",
    "review/time",
    "review/summary",
    "review/text"]

simple_header = [
    "productId",
    "userId",
    "profileName",
    "helpfulness",
    "score",
    "time",
    "summary",
    "text"]

infile = open(input_filepath, "rt", encoding="Latin-1")
csvfile = open(csv_filepath, "wt", encoding="UTF-8")


# Write a list of fields as a comma-separated row with a pipe (|) as a quote character.
def write_quoted_fields(csvfile, field_list):
    csvfile.write('|' + field_list[0] + '|')
    for field in field_list[1:]:
        csvfile.write(',|' + field + '|')
    csvfile.write("\n")


# Write the header line.
write_quoted_fields(csvfile, simple_header)

# Useful controls during debugging.
record_limit = 1000000
troublesome_records = [] #370, 211557, 226163, 519217, 521382, 525958, 531539

field_count = len(header)
line_count = 0
record_count = 0
currentLine = []
for line in infile:
    #print("Processing line: {}".format(line.strip()))
    line_count += 1
    line = line.strip()

    if line == "":
        if len(currentLine) == field_count:
            write_quoted_fields(csvfile, currentLine)
            record_count += 1

            if (record_count+1) in troublesome_records:
                print("[WARN] troublesome record -1: {}".format(currentLine))

            if record_count in troublesome_records:
                print("[WARN] troublesome record: {}".format(currentLine))

            if (record_count-1) in troublesome_records:
                print("[WARN] troublesome record +1: {}".format(currentLine))

        else:
            print("[WARN] current record appears to be incomplete: {}".format(currentLine))
        
        currentLine = []
        continue

    parts = line.split(": ", 1)
    
    # Check to see if the line looks sensible enough to be added.
    if len(parts) == 2:
        # If there are pipe characters in the text (unlikely), replace them with a slash.
        field = parts[1].strip().replace('|', '/')
        currentLine.append(field)
    else:
        # Throw this away - there are junk lines in the raw file, e.g.:
        # review/profileName: Sherry "Tell us about yourself!
        # School Princi...
        print("[WARN] only found {} parts after splitting line: {}".format(len(parts), parts))
        print("[WARN] line was: {}".format(line))
        
    if record_count > record_limit:
        break


# Write the final record (if it is complete).
if len(currentLine) == field_count:
    write_quoted_fields(csvfile, currentLine)
    record_count += 1
    
# Close files.
infile.close()
csvfile.close()

print("Finished - wrote {} lines and {} records.".format(line_count, record_count))

[WARN] only found 1 parts after splitting line: ['88 years old. ...']
[WARN] line was: 88 years old. ...
[WARN] only found 1 parts after splitting line: ['...creative powers b...']
[WARN] line was: ...creative powers b...
[WARN] only found 1 parts after splitting line: ['School Princi...']
[WARN] line was: School Princi...
[WARN] only found 1 parts after splitting line: ['School Princi...']
[WARN] line was: School Princi...
[WARN] only found 1 parts after splitting line: ['I am a voracious reader/li...']
[WARN] line was: I am a voracious reader/li...
[WARN] only found 1 parts after splitting line: ['School Princi...']
[WARN] line was: School Princi...
[WARN] only found 1 parts after splitting line: ['...creative powers b...']
[WARN] line was: ...creative powers b...
Finished - wrote 5116093 lines and 568454 records.


In [2]:
import numpy as np
import pandas as pd
from IPython.display import display

column_dtypes = {'productId': str, 'userId': str, 'profileName': str, 'helpfulness': str,
                 'score': np.float64, 'time': np.int64, 'summary': str, 'text': str}

# For this dataset, 'quoting' must be set to QUOTE_ALL (1) and the quotechar to a pipe (|).
# The problem is that values in some 'text' fields begin with a ", but don't end with one,
# and many review texts contain commas, unbalanced quotes and apostrophes.
review_df = pd.read_table(csv_filepath, delimiter=',', encoding="UTF-8", dtype=column_dtypes, 
                          quoting=1, quotechar='|', engine="c", skip_blank_lines=True, 
                          error_bad_lines=False, warn_bad_lines=True)

In [3]:
display(review_df['score'].value_counts())

# Convert score to an int, since it isn't truly a float.
review_df['score'] = review_df['score'].astype(int)

# Split helpfulness into 2 columns.

display(review_df.shape)
display(review_df.describe())
display(review_df.info())
display(review_df.head(10))

5.0    363122
4.0     80655
1.0     52268
3.0     42640
2.0     29769
Name: score, dtype: int64

(568454, 8)

,score,time
count,568454.000000,5.684540e+05
mean,4.183199,1.296257e+09
std,1.312259,4.804331e+07
min,1.000000,9.393408e+08
25%,4.000000,1.271290e+09
50%,5.000000,1.311120e+09
75%,5.000000,1.332720e+09
max,5.000000,1.351210e+09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 8 columns):
productId      568454 non-null object
userId         568454 non-null object
profileName    568438 non-null object
helpfulness    568454 non-null object
score          568454 non-null int32
time           568454 non-null int64
summary        568428 non-null object
text           568454 non-null object
dtypes: int32(1), int64(1), object(6)
memory usage: 32.5+ MB


None

,productId,userId,profileName,helpfulness,score,time,summary,text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0/0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0/0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0/0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1/1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0/0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [4]:
review_df['helpfulness_numerator'] = [x[0] for x in review_df['helpfulness'].str.split('/')]
review_df['helpfulness_denominator'] = [x[1] for x in review_df['helpfulness'].str.split('/')]

display(review_df.head(100))

,productId,userId,profileName,helpfulness,score,time,summary,text,helpfulness_numerator,helpfulness_denominator
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,1
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,0
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1,1
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,3,3
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0,0
5,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0/0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...,0,0
6,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0/0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...,0,0
7,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0/0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,0,0
8,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1/1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...,1,1
9,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0/0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...,0,0


In [5]:
review_df['date'] = pd.to_datetime(review_df['time'], unit='s')

display(review_df.dtypes)
display(review_df[['productId', 'userId', 'time', 'date']].head(10))

productId                          object
userId                             object
profileName                        object
helpfulness                        object
score                               int32
time                                int64
summary                            object
text                               object
helpfulness_numerator              object
helpfulness_denominator            object
date                       datetime64[ns]
dtype: object

,productId,userId,time,date
0,B001E4KFG0,A3SGXH7AUHU8GW,1303862400,2011-04-27
1,B00813GRG4,A1D87F6ZCVE5NK,1346976000,2012-09-07
2,B000LQOCH0,ABXLMWJIXXAIN,1219017600,2008-08-18
3,B000UA0QIQ,A395BORC6FGVXV,1307923200,2011-06-13
4,B006K2ZZ7K,A1UQRSCLF8GW1T,1350777600,2012-10-21
5,B006K2ZZ7K,ADT0SRK1MGOEU,1342051200,2012-07-12
6,B006K2ZZ7K,A1SP2KVKFXXRU1,1340150400,2012-06-20
7,B006K2ZZ7K,A3JRGQVEQN31IQ,1336003200,2012-05-03
8,B000E7L2R4,A1MZYO9TZK0BBI,1322006400,2011-11-23
9,B00171APVA,A21BT40VZCCYT4,1351209600,2012-10-26


In [11]:
del review_df['helpfulness']
review_df.to_csv("data/Reviews.csv",index=False)